In [1]:
# -*- coding: UTF-8 -*-

import pandas as pd
import numpy as np

# train_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/train_format1.csv', sep=',')
train_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/train_format1.csv', dtype=str)
print("train_format1 loaded successful")
print(train_df.head())

# user_log_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/user_log_format1.csv')
user_log_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/user_log_format1.csv', dtype=str)
print("user_log_format1 loaded successful")
print(user_log_df.head())

# user_info_df = pd.read_csv('/Users/Rosemary/Downloads/data_format1/user_info_format1.csv')
user_info_df = pd.read_csv('/Users/hanmufu/Workspaces/Repeat_Buyers_Prediction/data_format1/user_info_format1.csv', dtype=str)
print("user_info_format1 loaded successful")
print(user_info_df.head())

train_with_user_info = pd.merge(train_df, user_info_df)
print(train_with_user_info.head())

user_log_df['month'] = user_log_df['time_stamp'].str[:2]

train_format1 loaded successful
  user_id merchant_id label
0   34176        3906     0
1   34176         121     0
2   34176        4356     1
3   34176        2217     0
4  230784        4818     0
user_log_format1 loaded successful
  user_id  item_id cat_id seller_id brand_id time_stamp action_type
0  328862   323294    833      2882     2661       0829           0
1  328862   844400   1271      2882     2661       0829           0
2  328862   575153   1271      2882     2661       0829           0
3  328862   996875   1271      2882     2661       0829           0
4  328862  1086186   1271      1253     1049       0829           0
user_info_format1 loaded successful
  user_id age_range gender
0  376517         6      1
1  234512         5      0
2  344532         5      0
3  186135         5      0
4   30230         5      0
  user_id merchant_id label age_range gender
0   34176        3906     0         6      0
1   34176         121     0         6      0
2   34176        4356   

In [2]:
user_log_df = user_log_df.rename(columns={'seller_id': 'merchant_id'})

In [3]:
# add user age and gender into train_df
train_df = pd.merge(train_df, user_info_df, on=['user_id'])

In [4]:
# 计算user和merchant产生多少次action 十一月和之前几个月的均值的差
# Deviation of user's action between Nov and before
user_actions_at_nov = user_log_df.loc[user_log_df['month'] == "11"]
user_actions_not_at_nov = user_log_df.loc[user_log_df['month'] != "11"]
user_actions_not_at_nov = user_actions_not_at_nov.groupby(['user_id', 'merchant_id', 'month'], as_index=False)['month'].agg({'cnt':'count'})
user_actions_not_at_nov['user_id/merchant_id'] = user_actions_not_at_nov['user_id'] + '/' + user_actions_not_at_nov['merchant_id']
user_actions_not_at_nov = user_actions_not_at_nov.pivot(index='user_id/merchant_id', columns='month', values='cnt')
user_actions_not_at_nov = user_actions_not_at_nov.fillna(0)
user_actions_not_at_nov['mean_of_num_of_actions_not_in_nov'] = user_actions_not_at_nov.mean(axis=1)
user_actions_not_at_nov['user_id/merchant_id'] = user_actions_not_at_nov.index
user_actions_not_at_nov['user_id'] = user_actions_not_at_nov['user_id/merchant_id'].str.split('/').str[0]
user_actions_not_at_nov['merchant_id'] = user_actions_not_at_nov['user_id/merchant_id'].str.split('/').str[1]
user_actions_at_nov = user_actions_at_nov.groupby(['user_id', 'merchant_id', 'month'], as_index=False)['month'].agg({'cnt':'count'})
res = pd.merge(user_actions_at_nov, user_actions_not_at_nov, on=['user_id', 'merchant_id'])
res['user_action_deviation_between_nov_and_before'] = abs(res['cnt'] - res['mean_of_num_of_actions_not_in_nov'])
res = res.loc[:, ['user_id', 'merchant_id', 'user_action_deviation_between_nov_and_before']]
train_df = pd.merge(train_df, res, on=['user_id', 'merchant_id'])

In [5]:
# Number of user interactions with this merchant before Nov
user_actions_not_at_nov = user_log_df.loc[user_log_df['month'] != "11"]
user_actions_not_at_nov = user_actions_not_at_nov.groupby(['user_id', 'merchant_id'], as_index=False)['action_type'].agg({'num_of_interactions_before_nov':'count'})
train_df = pd.merge(train_df, user_actions_not_at_nov, on=['user_id', 'merchant_id'])

In [6]:
# 这个user和这个merchant的interaction 的均值和方差 （5-11月）
user_actions_at_nov = user_log_df.loc[user_log_df['month'] == "11"]
user_actions_not_at_nov = user_log_df.loc[user_log_df['month'] != "11"]
user_actions_not_at_nov = user_actions_not_at_nov.groupby(['user_id', 'merchant_id', 'month'], as_index=False)['month'].agg({'cnt':'count'})
user_actions_not_at_nov['user_id/merchant_id'] = user_actions_not_at_nov['user_id'] + '/' + user_actions_not_at_nov['merchant_id']
user_actions_not_at_nov = user_actions_not_at_nov.pivot(index='user_id/merchant_id', columns='month', values='cnt')
user_actions_not_at_nov = user_actions_not_at_nov.fillna(0)
user_actions_not_at_nov['user_id/merchant_id'] = user_actions_not_at_nov.index
user_actions_not_at_nov['user_id'] = user_actions_not_at_nov['user_id/merchant_id'].str.split('/').str[0]
user_actions_not_at_nov['merchant_id'] = user_actions_not_at_nov['user_id/merchant_id'].str.split('/').str[1]
user_actions_at_nov = user_actions_at_nov.groupby(['user_id', 'merchant_id', 'month'], as_index=False)['month'].agg({'cnt':'count'})
res = pd.merge(user_actions_at_nov, user_actions_not_at_nov, on=['user_id', 'merchant_id'])
res_for_mean_and_deviation = res.loc[:, ['cnt', '05', '06', '07', '08', '09', '10']]
res['mean_for_actions_num_from_may_to_nov'] = res_for_mean_and_deviation.mean(axis=1)
res['std_for_actions_num_from_may_to_nov'] = res_for_mean_and_deviation.std(axis=1)
res = res.loc[:, ['user_id', 'merchant_id', 'mean_for_actions_num_from_may_to_nov', 'std_for_actions_num_from_may_to_nov']]
train_df = pd.merge(train_df, res, on=['user_id', 'merchant_id'])

In [7]:
# res.index = res['user_id/merchant_id']

In [8]:
print(train_df.head())

  user_id merchant_id label age_range gender  \
0   34176        3906     0         6      0   
1  103296        2482     0         6      0   
2  104064        3870     0         6      1   
3  104064         669     0         6      1   
4  235392        1056     0         4      0   

   user_action_deviation_between_nov_and_before  \
0                                      5.166667   
1                                     10.833333   
2                                      6.333333   
3                                     16.000000   
4                                      5.666667   

   num_of_interactions_before_nov  mean_for_actions_num_from_may_to_nov  \
0                              29                              5.571429   
1                               1                              1.714286   
2                              10                              2.571429   
3                              18                              5.285714   
4                            

In [9]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
last_col = train_df.pop(train_df.columns[2])
train_df['label'] = last_col
train_df = train_df.drop(columns=['user_id', 'merchant_id'])

In [11]:
dataset = train_df.to_numpy()

In [12]:
# split data into X and y
X = dataset[:,0:6]
Y = dataset[:,6]

In [13]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [14]:
# fit model no training data
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [15]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)


In [20]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [value for value in y_pred]

In [21]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 89.83%
